基础爬虫
=======

In [4]:
# import lxml
import requests
from bs4 import BeautifulSoup
import re
import os

In [5]:
def html2soup(url):
    r = requests.get(url, timeout=30)
    r.encoding = 'utf-8'
    html = r.content
    soup = BeautifulSoup(html, 'lxml')
    return soup

def find_related_url(url, keyword):
    soup = html2soup(url)
    related_url = {}
    related = soup.find_all('div', attrs={'class': 'show74 left'})
    for u in related:
        if keyword in u.a.text:
            related_url[u.a.text] = u.a['href']
    return related_url

In [6]:
def url_list(main_site, start_pos, keyword, limit):
    depth = 0
    start_url = "".join([main_site, start_pos])
    all_related = find_related_url(start_url, keyword)
    while depth < limit:
        print('Search Time:', depth)
        print('Current URLs:', all_related)
        depth = depth+1
        for pos in all_related.values():
            _ = "".join([main_site, pos])
            new_url = find_related_url(_, keyword)
            all_related = {**all_related, **new_url}
    return all_related

In [7]:
def get_content(url):
    soup = html2soup(url)
    # three contents: title, main body, right columns
    title = soup.find('div', attrs={'class': 'show17'}).text.strip()
    _body = soup.find('div', attrs={'class': 'show63'}).text.strip()
    _body = respace(_body)
    body = "\n[来源]({})\n\n{}".format(url, _body)
    print('Finish parsing')
    return {'title': title, 'body': body}

In [31]:
def respace(text):
    text = text.replace('\n', '')
    text = text.replace('　', '')
    text = text.replace(' ', '')
    text = text.replace('。','。\n')
    return text

In [21]:
def writeFile(content_dict, folder='Storage'):
    if not os.path.exists(folder):
        os.mkdir(folder)
    with open('{}\\{}.md'.format(folder, content_dict['title']), 'w', encoding='utf-8') as f:
        f.write("{}\n=====\n".format(content_dict['title']))
        f.write("{}".format(content_dict['body']))
    print('Finish writing 《{}》'.format(content_dict['title']))

In [32]:
if __name__ == '__main__':
    main_site = "http://www.cnbridge.cn/"
    start_pos = "2017/1208/551262.html"
    keyword = "BIM"
    time_limit = 1
    urls = url_list(main_site, start_pos, keyword, time_limit)
    for _ in urls.values():
        writeFile(get_content("".join([main_site, _])), folder='BrIM')

Search Time: 0
Current URLs: {'BIM 在国外应用的现状研究': '2017/1208/551262.html', '国内外BIM标准发展情况': '2017/1208/551227.html', '国内外BIM主流软件包括哪些？': '2017/1208/551221.html', '中国BIM，世界BIM': '2017/1208/551220.html', '国外公司公路BIM解决方案存在的问题': '2017/1208/551219.html', 'BIM国外应用介绍之新加坡': '2017/1208/551201.html', 'BIM在美国的应用现状': '2017/1208/551200.html'}
Finish parsing
Finish writing 《BIM 在国外应用的现状研究》
Finish parsing
Finish writing 《国内外BIM标准发展情况》
Finish parsing
Finish writing 《国内外BIM主流软件包括哪些？》
Finish parsing
Finish writing 《中国BIM，世界BIM》
Finish parsing
Finish writing 《国外公司公路BIM解决方案存在的问题》
Finish parsing
Finish writing 《BIM国外应用介绍之新加坡》
Finish parsing
Finish writing 《BIM在美国的应用现状》
